In [1]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm    # 03. 導入自訂的HandTrackingModule.py
# 07. 匯入sleep
from time import sleep
# 09. 匯入鍵盤控制
from pynput.keyboard import Controller
from pynput.keyboard import Key, Controller

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 1280, 720

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. FPS時間設定
pTime = 0

# 03. 使用 HandTrackingModule.py函式擷取手部資訊
detector = htm.handDetector(detectionCon=0.75, maxHands=1)
# 08. 設定顯示透明度
def drawAll(img, buttonList):
    imgNew = np.zeros_like(img, np.uint8)
    for button in buttonList:
        button.draw(imgNew)
    
    out = img.copy()
    alpha =0.2
    mask = imgNew.astype(bool)
    out[mask] =  cv2.addWeighted(img, alpha, imgNew, 1-alpha, 0)[mask]
    return out

# 04. 在畫面中建立一組虛擬鍵盤按鈕
class Button():
    def __init__(self, pos, text, size=[85,85]):
        self.pos = pos
        self.size = size
        self.text = text
    
    def draw(self, img,ccolors=(255,0,255), tcolors=(0,255,255)):
        #print(self.text)
        if self.text == "black" or self.text == "enter":
            x, y = self.pos
            w, h = self.size
            cv2.rectangle(img, self.pos, (x+w, y+h), tcolors, thickness=10)
            cv2.rectangle(img, self.pos, (x+w, y+h), ccolors, cv2.FILLED)
            if self.text == "black":
                cv2.line(img,(x+20,y+h//2),(x+80,y+h//2),(0, 0, 255), 5)
                cv2.line(img,(x+20,y+h//2),(x+40,y+20),(0, 0, 255), 5)
                cv2.line(img,(x+20,y+h//2),(x+40,y+60),(0, 0, 255), 5)
            if self.text == "enter":
                cv2.line(img,(x+20,y+h//2),(x+80,y+h//2),(0,100, 255), 5)
                cv2.line(img,(x+20,y+h//2),(x+40,y+20),(0, 100, 255), 5)
                cv2.line(img,(x+20,y+h//2),(x+40,y+60),(0, 100, 255), 5)
                cv2.line(img,(x+80,y+h//2),(x+80,y),(0, 100, 255), 5)
        else:
            x, y = self.pos
            w, h = self.size
            cv2.rectangle(img, self.pos, (x+w, y+h), tcolors, thickness=10)
            cv2.rectangle(img, self.pos, (x+w, y+h), ccolors, cv2.FILLED)
            cv2.putText(img, self.text, (self.pos[0]+18, self.pos[1]+68), cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)
        #cv2.line(img, 開始座標, 結束座標, 顏色, 線條寬度)
        return img
# 04.宣告一組虛擬鍵盤按鈕
keys=[("0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "black"),
      ("Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"), 
      ("A", "S", "D", "F", "G", "H", "J", "K", "L", ":", "enter"),
      ("Z", "X", "C", "V", "B", "N", "M", ",", ".", "/")]
buttonList = []
for n in range(0, 3):
    for x, key in enumerate(keys[n]):
        buttonList.append(Button([100*x+50, 100*n+100], key))
# 07. 宣告輸入字串
finalText = ""
# 09. 鍵盤控制
keyboard = Controller()
keyboard.press(Key.backspace)
keyboard.release(Key.backspace)
keyboard.press(Key.enter)
keyboard.release(Key.enter)
while True:
    success, img = cap.read()
    # 水平翻轉畫面
    img = cv2.flip(img, 1) 
    # 03. 偵測手並傳回座標位置
    img = detector.findHands(img, draw = False)
    # 04. 在畫面中建立一組虛擬鍵盤按鈕
    #for button in buttonList:
        #img = button.draw(img)
    img = drawAll(img, buttonList)  # 08. 設定透明鍵盤

    lmList, bbox = detector.findPosition(img)
    # 05. 檢查手指是否在某一個按鈕位置
    if lmList:
        for button in buttonList:
            x, y = button.pos
            w, h = button.size
            if (x < lmList[8][1] < x+w) and (y < lmList[8][2] < y+h):
                img = button.draw(img, ccolors=(0, 255, 0))
                # 06. 點擊模式:食指與中指併攏
                length, _, _ = detector.findDistance(8, 12, img)
                cv2.putText(img, f'L:{int(length)}', (180, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 3)
                #print(length)
                # 07. 兩指距離短則輸入字元
                if length < 50:
                    # 09. 鍵盤控制
                    keyboard = Controller()
                    keyboard.press(Key.backspace)
                    keyboard.release(Key.backspace)
                    sleep(0.15)
                    keyboard.press(Key.enter)
                    keyboard.release(Key.enter)
                    sleep(0.1)
                    img = button.draw(img, ccolors=(0, 255, 0)) 
                    finalText += button.text 
                    sleep(0.15)
    # 07. 顯示文字
    cv2.rectangle(img, (50, 550), (1000, 650), (175, 255, 175), cv2.FILLED)
    cv2.putText(img, finalText, (60, 600), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 3)
    # 02. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)


    cv2.imshow("Image", img)
    # 01. 按Q離
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


In [21]:
import cv2 as cv 
import mediapipe as mp
import numpy as np
from time import sleep
from pynput.keyboard import Controller, Key

class displaySolutionSet():
    def __init__(self, capture, width, height):
        self.capture = capture
        self.width = width
        self.height = height
        
    def solution_set(self):
        self.capture.set(3, self.width)
        self.capture.set(4, self.height)
keyText=[("0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "black"),
         ("Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"), 
         ("A", "S", "D", "F", "G", "H", "J", "K", "L", ":", "enter"),
         ("Z", "X", "C", "V", "B", "N", "M", ",", ".", "/")]
keyboard = Controller()
def translucent(image, post1, post2, BChannel=0, GChannel=255, RChannel=0, value=0.8):
    channel = [BChannel, GChannel, RChannel]
    for i in range(3):
        image[post1[1]: post2[1], post1[0]:post2[0], i] = \
             image[post1[1]: post2[1], post1[0]:post2[0], i]*value = channel[i]*(1-value)
    return image
def draw_keyboard(image, rectPos1 = (50, 50), rectPos2 = (120, 120), textPos = (62, 107), textColor=(255, 255, 255)):
    for i, list in enumerate(keyText):
        for j, l in enumerate(list):
            if i == 2 and j == 10:
                image= translucent(image, (rectPos1[0]=90*j, rectPos1[1]&#43;90*i), (rectPos2[0]&#43;90*(j&#43;1), rectPos2[1]&#43;90*i), value&#61;0.7)
                cv.putText(image, l, (textPos[0]&#43;90*j, textPos[1]&#43;90*i), 
                           fontFace&#61;cv.FONT_HERSHEY_PLAIN, fontScale&#61;2, color&#61;textColor, thickness&#61;2)
            elif i&#61;&#61; 3 and j &#61;&#61; 10:
                image &#61; translucent(image, (rectPos1[0]&#43;90*j, rectPos1[1]&#43;90*i), (rectPos2[0]&#43;90*(j&#43;1), rectPos2[1]&#43;90*i), value&#61;0.7)
                cv.putText(image, l, (textPos[0]&#43;90*j, textPos[1]&#43;90*i), 
                           fontFace&#61;cv.FONT_HERSHEY_PLAIN, fontScale&#61;2, color&#61;textColor, thickness&#61;2)
            else:
                image &#61; translucent(image, (rectPos1[0]&#43;90*j, rectPos1[1]&#43;90*i), (rectPos2[0]&#43;90*j, rectPos2[1]&#43;90*i), value&#61;0.7)
                cv.putText(image, l, (textPos[0]&#43;90*j, textPos[1]&#43;90*i), 
                           fontFace&#61;cv.FONT_HERSHEY_PLAIN, fontScale&#61;3, color&#61;textColor, thickness&#61;2